In [1]:
import getpass
import json
import uuid

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import (check_premises,
                                   execute_query,
                                   execute_query_pipe,
                                   apply_rule,
                                   get_rule_parameters)

In [2]:
LOCAL_TOKEN = True 


def allocate_forge_session(org, project):
    config_path = "./config.yaml"
    ENDPOINT = "https://bbp.epfl.ch/nexus/v1"  

    if LOCAL_TOKEN: 
        with open('token.txt') as f:
            TOKEN = f.read()
    else: 
        TOKEN = getpass.getpass()
        
    searchendpoints = {
        "sparql": {
           "endpoint": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
       }
    }
    
    return KnowledgeGraphForge(
        config_path,
        endpoint=ENDPOINT,
        token=TOKEN, 
        bucket=f"{org}/{project}", 
        searchendpoints=searchendpoints,
        debug=True
    )

In [3]:
#                 ?id objectOfStudy.id $PathQueryParameter .

q = {
        "type": "SparqlQuery",
        "hasBody": 
        """
            SELECT ?id ?br
            WHERE { 
                ?id $whatever .
                ?id nsg:brainLocation/nsg:brainRegion ?br .
                }
        """,
        "hasParameter": [
            {
                "type": "MultiPredicateObjectPair",
                "description": "paths to the properties being checked",
                "name": "whatever"
            },
#             {
#                 "type": "uri",
#                 "description": "some other passed param that should still exist",
#                 "name": "PathQueryParameter"
#             },
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "bbp",
            "project": "atlas"
        },
        "resultParameterMapping": [
            {
                "parameterName": "entityBr",
                "path": "br"
            },
            {
                "parameterName": "entity",
                "path": "id"
            }
        ]
    }

input_filters = {
#     "PathQueryParameter": "<http://bbp.epfl.ch/neurosciencegraph/taxonomies/objectsofstudy/singlecells>",
    "whatever": [
        (("rdf:type", "path"), ("<https://neuroshapes.org/NeuronMorphology>", "uri")), 
        (("contribution/agent", "path"), ("<https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028>", "uri"))
    ]   
}

res = execute_query(allocate_forge_session, q, input_filters, last_query=True)

In [5]:
len(res)

331